In [ ]:
import glob
import pandas as pd
import re
import ipywidgets as widgets


In [ ]:
# Read all csv files
files = glob.glob(f'*.csv')
files.insert(0, '')


In [ ]:
data_regex = re.compile("(\d+) (\w+) (\d+) \(\w+\),")
meta_regex = re.compile("(ATM)")
columns = ['Date', 'Profit', 'Max Profit/Loss', 'BN Gap Up/Down', 'India VIX',
           'BN Prev Day H/L', 'Spot Change', '(1) BN Buy 1 Lot ATM PE', '(2) BN Buy 1 Lot ATM CE']


In [ ]:
# Define widgets
strategy_dropdown = widgets.Dropdown(
    options=files,
    description='Strategy:'
)

meta_output = widgets.Output()
aggregate_output = widgets.Output()


def strategy_dropdown_handler(change):
    file = change.new
    meta_output.clear_output()
    aggregate_output.clear_output()

    if file == '':
        return

    file_reader = open(file, 'r')
    lines = file_reader.readlines()
    df = pd.DataFrame(list(filter(data_regex.match, lines)))
    df = df[0].str.split(",", expand=True)
    df.columns = columns
    df['Profit'] = pd.to_numeric(df['Profit'])
    df['Day'] = df['Date'].str.extract('\((\w+)\)', expand=True)
    df['Month'] = df['Date'].str.extract('\d+ (\w+) \d+', expand=True)
    df['Year'] = df['Date'].str.extract('\d+ \w+ (\d+)', expand=True)

    meta = "".join(list(filter(meta_regex.match, lines)))

    df_agg = pd.pivot_table(df, values='Profit', columns=['Day'], index=[
                            'Year', 'Month'], aggfunc='sum', margins=True)

    with meta_output:
        display(print(meta))

    with aggregate_output:
        display(df_agg)


strategy_dropdown.observe(strategy_dropdown_handler, names='value')


In [ ]:
# Dropdown selector widget
display(strategy_dropdown)


In [ ]:
# File meta information
display(meta_output)


In [ ]:
# Aggregated by day/month/year
display(aggregate_output)
